In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import json

In [ ]:
!pip install scrapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.4/286.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.1/247.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 7.0 MB/s eta 0:00:00


In [ ]:
!scrapy startproject tutorial

In [ ]:
!scrapy crawl quotes

In [ ]:
%cd /content/drive/MyDrive/CS114/tutorial

/content/drive/MyDrive/CS114/tutorial


In [ ]:
import scrapy

class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://timkiem.vnexpress.net/?q=t%E1%BB%99i%20%C3%A1c&media_type=all&fromdate=&todate=&latest=&cate_code=&search_f=title,tag_list&date_format=year&page=2'
        ]
        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        allLinks = response.css('h3.title-news a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = response.css('a.btn-page next-page::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        title = response.css('.title-detail::text').get()
        title = title.replace('\n', '')
        title = title.replace('\t', '')
        paragraph = response.css('.fck_detail p::text').getall()
        text = ''
        count = 0
        for sentence in paragraph:
            if count == 4:
                break
            sentence = sentence.replace('\xa0', '')
            text = text + sentence
            count += 1
        yield {
            'title': title,
            'text': text,
            'is_negative': 1
        }

In [ ]:
!scrapy crawl news -o dataset-toiac1.json

2024-01-11 08:12:57 [scrapy.utils.log] INFO: Scrapy 2.11.0 started (bot: tutorial)
2024-01-11 08:12:57 [scrapy.utils.log] INFO: Versions: lxml 4.9.4.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0], pyOpenSSL 23.3.0 (OpenSSL 3.1.4 24 Oct 2023), cryptography 41.0.7, Platform Linux-6.1.58+-x86_64-with-glibc2.35
2024-01-11 08:12:57 [scrapy.addons] INFO: Enabled addons:
[]
2024-01-11 08:12:57 [asyncio] DEBUG: Using selector: EpollSelector
2024-01-11 08:12:57 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2024-01-11 08:12:57 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2024-01-11 08:12:57 [scrapy.extensions.telnet] INFO: Telnet Password: 4a2cb67058ef21f2
2024-01-11 08:12:58 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 's

In [ ]:
data = pd.read_json('/content/drive/MyDrive/CS114/tutorial/dataset-toiac1.json')
data

,title,text,is_negative
0,Bị kiểm soát chặt vì chồng tin những lời đồn á...,"Tôi 33 tuổi, có gia đình và hai con nhỏ, thu n...",1
1,Sát thủ điều khiển đàn chó dữ phi tang tội ác,"Tháng 1/1995, đồn cảnh sát thành phố Ngân Xuyê...",1
2,LHQ không phát hiện hành động diệt chủng ở Ukr...,"""Chúng tôi không ghi nhận việc đã xảy ra một c...",1
3,Tội ác khi người tình nói 'không thể sống chung',Hành vi của Hiệp được nêu trong cáo trạng VKSN...,1
4,Mẹ gánh nỗi oan giết con từ tội ác của đàn chó...,Ayers Rock là khối đá sa thạch màu đỏ lớn ở mi...,1
5,Tội ác sau lời thách đố 'chứng minh tình yêu',"Sáng 10/2/2020, Đội trưởng Cứu hỏa địa phương ...",1
6,50 ngày gieo rắc kinh hoàng của 'bộ đôi ác quỷ',Đường Diệm Bình sinh năm 1962 tại huyện Thanh ...,1
7,Iran dọa phong tỏa Địa Trung Hải,"""Họ sẽ sớm chứng kiến Địa Trung Hải, eo biển G...",1
8,Tội ác của giám đốc bốn lần thuê người giết vợ,"Tối 25/5/2009, một thanh niên ở thôn Bắc Quan,...",1
9,Tội ác của thiếu niên si mê bà mẹ đơn thân,"Rạng sáng 6/3/2002, cậu bé Coty, 9 tuổi, nhờ n...",1


In [ ]:
import json
import os

# List path to combine
file_paths = ['/content/drive/MyDrive/CS114/tutorial/dataset-thoisu.json', '/content/drive/MyDrive/CS114/tutorial/dataset-tainangiaothong2.json', '/content/drive/MyDrive/CS114/tutorial/dataset-tainangiaothong.json',
              '/content/drive/MyDrive/CS114/tutorial/dataset-phapluat2.json','/content/drive/MyDrive/CS114/tutorial/dataset-phapluat.json',
              '/content/drive/MyDrive/CS114/tutorial/dataset-giaothong.json','/content/drive/MyDrive/CS114/tutorial/dataset-giaoduc.json','/content/drive/MyDrive/CS114/tutorial/dataset-baoluc.json']

# Final Path
output_file_path = '/content/drive/MyDrive/CS114/FinalDataset.json'

Final_Dataset = []

for file_path in file_paths:
    with open(file_path, 'r') as file:
        data = json.load(file)
        Final_Dataset += data

# Save
with open(output_file_path, 'w') as output_file:
    json.dump(Final_Dataset, output_file, indent=2)
